In [3]:
#6.4 Speeding up optimization with the full Platt SMO
class optStruct:
    def __init__(self, dataMatIn, classLabels, C, toler):
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = shape(dataMatIn)[0]
        self.alphas = mat(zeros((self.m, 1)))
        self.b = 0
        self.eCache = mat(zeros((self.m, 2)))
        
def clacEk(oS, k):
    fXk = float(multiply(oS.alphas, oS.labelMat).T * (oS.X * oS.X[k, :],T)) + oS.b
    Ek = fXk - float(oS.labelMat[k])
    return Ek

def selectJ(i, oS, Ei):
    """
    内循环启发方式2
    Parameters：
        i - 标号为i的数据的索引值
        oS - 数据结构
        Ei - 标号为i的数据误差
    Returns:
        j, maxK - 标号为j或maxK的数据的索引值
        Ej - 标号为j的数据误差
    """
    maK, maxDeltaE, Ej = -1, 0, 0
    #根据Ei更新误差缓存
    #Ei保存的两维中，第一维表示Ei的有效性，第二维才是真实值
    oS.eCache[i] = [1, Ei]
    #返回误差不为0的数据的索引值
    #.A转化为array类型，得到想要的非零的向量的行号
    validEcacheList = nonzero(oS.eCache[:, 0].A)[0]
    #如果有不为0的误差
    if (len(validEcacheList)) > 1:
        for k in validEcacheList:
            if k == i:
                continue
            Ek = calcEk(oS, k)
            deltaE = abs(Ei - Ek)
            #找到maxDeltaE
            if (deltaE > maxDeltaE):
                maK, maxDeltaE, Ej = k, deltaE, Ek
        return maxK, Ej
    #没有不为0的误差
    else:
        #随机选择alpha_j的索引值
        j = selectJrand(i, oS.m)
        Ej = calcEk(oS, j)
    return j, Ej

def updateEk(oS, k):
    Ek = calcEk(oS, k)
    oS.eCache[k] = [1, Ek]

In [4]:
def innerL(i, oS):
    Ei = calcEk(oS, i)
    #判断y*f(x)的取值
    if ((oS.labelMat[i] * Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or\
    ((oS.labelMat[i] * Ei > oS.tol) and (oS.alphas[i] > 0)):
        j, Ej = selectJ(i, oS, Ei)
        alphaIold = oS.alphas[i].copy()
        alphaJold = oS.alphas[j].copy()
        #y(label)一个为+1，一个为-1
        if (oS.labelMat[i] != oS.labelMat[j]):
            L = max(0, oS.alphas[j] - oS.alphas[i])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        #两个label相同
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])
        if L == H: print('L == H'); return 0
        eta = 2.0 * oS.X[i,:] * oS.X[j, :].T - oS.X[i,:] * oS.X[i,:].T - oS.X[j,:] * oS.X[j,:].T
        if eta >= 0: print('eta >= 0'); return 0
        oS.alphas[j] -= oS.labelMat[j] * (Ei - Ej) / eta
        oS.alphas[j] = clipAlpha(oS.alphas[j], H, L)
        updateEk(oS, j)
        if (abs(oS.alphas[j] - alphaJold) < 0.00001):
            print('j not moving enough')
            return 0
        oS.alphas[i] += oS.labelMat[j] * oS.labelMat[i] * (alphaJold - oS.alphas[j])
        updateEk(oS, i)
        b1 = oS.b - Ei - oS.labelMat[i] * (oS.alphas[i] - alphaIold) *\
        oS.X[i,:] * oS.X[i,:].T - oS.labelMat[j] * \
        (oS.alphas[j] - alphaJold) * oS.X[i,:] * oS.X[j,:].T
        b2 = oS.b - Ej - oS.labelMat[i] * (oS.alphas[i] - alphaIold) *\
        oS.X[i,:] * oS.X[j,:].T - oS.labelMat[j] * \
        (oS.alphas[j] - alphaJold) * oS.X[j,:] * oS.X[j,:].T
        if (0 < oS.alphas[i]) and (oS.C > oS.alphas[i]): oS.b = b1
        elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]): oS.b = b2
        else: oS.b = (b1 + b2) / 2.0
        return 1
    else:
        return 0

In [5]:
def smoP(dataMatIn, classLabels, C, toler, maxIter, kTup = ('lin', 0)):
    """
    完整的线性SMO算法
    Parameters：
        dataMatIn - 数据矩阵
        classLabels - 数据标签
        C - 松弛变量
        toler - 容错率
        maxIter - 最大迭代次数
        kTup - 包含核函数信息的元组
    Returns:
        oS.b - SMO算法计算的b
        oS.alphas - SMO算法计算的alphas
    """
    #Platt SMO算法是通过一个外循环来选择第一个alpha值的，并且其选择过程会在两种方式之间进行交替：
    #一种方式是在所有数据集上进行单遍扫描，另一种方式则是在非边界alpha中实现单遍扫描。　　
    #所谓非边界alpha指的就是那些不等于边界0或者C的alpha值。对整个数据集的扫描相当容易，而实现非边界alpha值的扫描时，
    #首先需要建立这些alpha值的列表，然后再对这个表进行遍历。同时，该步骤会跳过那些已知的不会改变的alpha值，即。
    #在选择第一个alpha值后，算法会通过一个内循环来选择第二个alpha值。在优化过程中，会通过最大化步长的方式来获得第二个alpha值。
    #在简化版SMO算发放中，我们会在选择j之后计算错误率Ej。
    #但在这里，我们会建立一个全局的缓存用于保存误差值，并从中选择使得步长或者说Ei-Ej最大的alpha值。
    #初始化数据结构
    oS = optStruct(mat(dataMatIn), mat(classLabels).transpose(), C, toler)
    #初始化当前迭代次数
    iter = 0
    entireSet = True
    alphaPairsChanged = 0
    #遍历整个数据集alpha都没有更新或者超过最大迭代次数,则退出循环
    while (iter < maxIter) and ((alphaPairsChanged > 0) or (entireSet)):
        alphaPairsChanged = 0
        #遍历整个数据集
        if entireSet:
            for i in range(oS.m):
                #使用优化的SMO算法
                alphaPairsChanged += innerL(i, oS)
            print('fullSet, iter: %d i:%d, pairs changed %d' %(iter, i, alphaPairsChanged))
            iter += 1
        #遍历非边界值
        else:
            #遍历不在边界0和C的alpha
            nonBoundIs = nonzero((oS.alphas.A > 0) * (oS.alphas.A < C))[0]
            for i in nonBoundIs:
                alphaPairsChanged += innerL(i, oS)
                print('non-bound, iter: %d i:%d, pairs changed %d' % (iter, i, alphaPairsChanged))
            iter +=1
        #遍历一次后改为非边界遍历
        if entireSet: entireSet = False
        elif (alphaPairsChanged == 0): entireSet = True
        print('iteration number: %d' % iter)
    #返回SMO算法计算的b和alphas
    return iS.b, oS.alphas

In [6]:
dataArr, labelArr = loadDataSet('testSet.txt')

NameError: name 'loadDataSet' is not defined